In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pprint import pprint
import requests
import time
from scipy.stats import linregress
import os
import csv

# Import API key
#from api_keys import api_key_file

# Output File (CSV)
output_data_file = "output_data/duistats.csv"

# File to Load (**Remember to Change These file names to DUI saved file names)
dui_2013_data = "nrs_2013_combined.csv"
dui_2007_data = "nrs_2007_final.csv"

# Read DUI 2007 and 2013 Data Files and store into Pandas DataFrames(can't start with a number)
dui2013_data = pd.read_csv(dui_2013_data)
dui2007_data = pd.read_csv(dui_2007_data)



C:\Users\eliza\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (22,24,29,30,33,35,36,37,53,57,73,74,148,149,150,151,152,153,154,155,170,171,172,173,174,175,176,177,195,217,219,904,913,914,927,928,929,930,931,932,933,934,935,961) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\eliza\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (31,32,35,38,46,47,195,216,217,260,262,267,269,281,282,292,295,298,299,312) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [24]:
dui2013_data

,protocol49_only_NEW_wgt,NEW_FINAL_weights,npsu13,PRE_1,lg10alc_fixed,PGR_1,PRE_2,din,data_session_original,data_session_n,...,blood_class_synth_2007,blood_class_2007_allTHC,of_blood_class_2007,of_marijuana_all_07,blood_marijuana_all_07,of_blood_marijuana_all_07,of_blood_THC_pos_13,of_marijuana_all_13,blood_marijuana_all_13,of_blood_marijuana_all_13
0,0.22557,0.239149,1195,#NULL!,#NULL!,#NULL!,#NULL!,03 - CA - 50 - 3 - 01,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,#NULL!,NaN,NaN,NaN
1,0.22557,0.239149,1195,#NULL!,#NULL!,#NULL!,#NULL!,03 - CA - 50 - 3 - 02,3,3,...,NaN,NaN,Negative,Negative,NaN,Negative,0,Negative,NaN,Negative
2,0.22557,0.239149,1195,#NULL!,#NULL!,#NULL!,#NULL!,03 - CA - 50 - 3 - 03,3,3,...,NaN,NaN,Stimulan,Other drugs,NaN,Other drugs,0,Other drugs,NaN,Other drugs
3,0.22557,0.239149,1195,#NULL!,#NULL!,#NULL!,#NULL!,03 - CA - 50 - 3 - 04,3,3,...,Negative,Negative,Negative,Negative,Negative,Negative,0,Negative,Negative,Negative
4,0.22557,0.239149,1195,#NULL!,#NULL!,#NULL!,#NULL!,03 - CA - 50 - 3 - 05,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,#NULL!,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11317,0.00000,0.256170,1195,#NULL!,#NULL!,#NULL!,#NULL!,72 - UT - 56 - 4 - 07,4,4,...,Negative,Negative,Negative,Negative,Negative,Negative,0,Negative,Negative,Negative
11318,0.00000,0.235396,1195,#NULL!,#NULL!,#NULL!,#NULL!,72 - UT - 56 - 5 - 01,5,5,...,Negative,Negative,Negative,Negative,Negative,Negative,0,Negative,Negative,Negative
11319,0.00000,0.235396,1195,#NULL!,#NULL!,#NULL!,#NULL!,72 - UT - 56 - 5 - 02,5,5,...,Negative,Negative,Negative,Negative,Negative,Negative,0,Negative,Negative,Negative
11320,0.00000,0.235396,1195,#NULL!,#NULL!,#NULL!,#NULL!,72 - UT - 56 - 5 - 03,5,5,...,Negative,Negative,Negative,Negative,Negative,Negative,0,Negative,Negative,Negative


In [29]:
#renames for merge
dui2007_data.rename(columns = {"STATE": "State", "NRS_ZIPCODE": "Zipcode", "DATE":"Date","session": "Day or Night?","NRS_RACE":"Race", 
                               "NRS_DSEX":"Gender", "NRS_AGE":"Age Range","NRS_SCHOOL": "Education Level","NRS_EMPLOY": "Employed", 
                               "NRS_VTYPE": "Vehicle Type", "NRS_FROM": "From Where", "NRS_HEADED": "To Where","NRS_BETMI": "Trip distance", 
                               "NRS_TODRK": "Alcohol Today", "resulbac2": "Blood Alcohol Content"}, inplace = True)



In [28]:
dui2007_data.add_suffix("_2007")

,DIN_2007,Day or Night?_2007,SESSION1_2007,timeday_2007,State_2007,NRS_AREA_2007,Date_2007,psu_2007,eligible_2007,nrsagree_r_2007,...,bac08_2007,AUD_OFFERED_2007,AUD_CURRENT_2007,AUD_COMPLETE_2007,AUD_DEPEND_2007,AUD_ABUSE_2007,AUD_HEAVYD_2007,AUD_BINGE_2007,AUD_NORMATIVE_2007,AUD_CAT_2007
0,004/36/3/006,3,Session # 3 = Fri. NIGHT 2nd site,Nighttime,TN,Memphis [CITY],11/3/2007,36,1,2,...,0,0,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,NaN
1,005/17/1/007,1,Session # 1 = Friday Day,Day time,IL,Cook County (excluding city o,NaN,17,1,2,...,#NULL!,0,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,NaN
2,005/18/1/009,1,Session # 1 = Friday Day,Day time,IN,Lake County,NaN,18,1,999,...,0,0,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,NaN
3,007/28/1/005,1,Session # 1 = Friday Day,Day time,MI,Wayne County (excluding city,10/12/2007,28,1,2,...,0,0,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,NaN
4,008/15/5/003,5,Session # 5 = Sat. NIGHT 2nd site,Nighttime,IA,Floyd & Howard Counties,NaN,15,1,2,...,#NULL!,0,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11115,053/18/2/003,2,Session # 2 = Fri. NIGHT 1st site,Nighttime,IN,Lake County,NaN,18,1,1,...,0,1,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,NaN
11116,010/46/3/006,3,Session # 3 = Fri. NIGHT 2nd site,Nighttime,PA,Delaware County,NaN,46,1,1,...,0,1,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,NaN
11117,039/13/5/007,5,Session # 5 = Sat. NIGHT 2nd site,Nighttime,FL,Fort Lauderdale & Hollywood,NaN,13,1,1,...,0,1,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,NaN
11118,040/46/1/002,1,Session # 1 = Friday Day,Day time,PA,Delaware County,NaN,46,1,1,...,0,1,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,NaN


In [30]:
dui2013_data.rename(columns = {"MyZipCodeIs": "Zipcode", "Time_Stamp":"Date","data_session_original": "Day or Night?","race_n":"Race", 
                               "DriverAgeYears":"Age Range","EducationLevel": "Education Level","AreYouEmployed": "Employed", 
                               "VehicleType": "Vehicle Type", "FromWhere": "From Where", "ToWhere": "To Where","MilesTraveling": "Trip distance", 
                               "AlcoholToday": "Alcohol Today", "bac_from_blood": "Blood Alcohol Content"}, inplace = True)


In [31]:
dui2007_data.add_suffix("_2013")

,DIN_2013,Day or Night?_2013,SESSION1_2013,timeday_2013,State_2013,NRS_AREA_2013,Date_2013,psu_2013,eligible_2013,nrsagree_r_2013,...,bac08_2013,AUD_OFFERED_2013,AUD_CURRENT_2013,AUD_COMPLETE_2013,AUD_DEPEND_2013,AUD_ABUSE_2013,AUD_HEAVYD_2013,AUD_BINGE_2013,AUD_NORMATIVE_2013,AUD_CAT_2013
0,004/36/3/006,3,Session # 3 = Fri. NIGHT 2nd site,Nighttime,TN,Memphis [CITY],11/3/2007,36,1,2,...,0,0,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,NaN
1,005/17/1/007,1,Session # 1 = Friday Day,Day time,IL,Cook County (excluding city o,NaN,17,1,2,...,#NULL!,0,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,NaN
2,005/18/1/009,1,Session # 1 = Friday Day,Day time,IN,Lake County,NaN,18,1,999,...,0,0,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,NaN
3,007/28/1/005,1,Session # 1 = Friday Day,Day time,MI,Wayne County (excluding city,10/12/2007,28,1,2,...,0,0,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,NaN
4,008/15/5/003,5,Session # 5 = Sat. NIGHT 2nd site,Nighttime,IA,Floyd & Howard Counties,NaN,15,1,2,...,#NULL!,0,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11115,053/18/2/003,2,Session # 2 = Fri. NIGHT 1st site,Nighttime,IN,Lake County,NaN,18,1,1,...,0,1,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,NaN
11116,010/46/3/006,3,Session # 3 = Fri. NIGHT 2nd site,Nighttime,PA,Delaware County,NaN,46,1,1,...,0,1,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,NaN
11117,039/13/5/007,5,Session # 5 = Sat. NIGHT 2nd site,Nighttime,FL,Fort Lauderdale & Hollywood,NaN,13,1,1,...,0,1,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,NaN
11118,040/46/1/002,1,Session # 1 = Friday Day,Day time,PA,Delaware County,NaN,46,1,1,...,0,1,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,#NULL!,NaN


In [22]:
# Combine the data into a single dataset.  
dui_data_complete = pd.merge(dui2013_data,dui2007_data, how="left", on="State")

MemoryError: Unable to allocate 43.7 GiB for an array with shape (962, 6100920) and data type object

In [21]:
dui_data_complete.head()

,protocol49_only_NEW_wgt,NEW_FINAL_weights,npsu13,PRE_1,lg10alc_fixed,PGR_1,PRE_2,din,data_session_original,data_session_n,...,bac08,AUD_OFFERED,AUD_CURRENT,AUD_COMPLETE,AUD_DEPEND,AUD_ABUSE,AUD_HEAVYD,AUD_BINGE,AUD_NORMATIVE,AUD_CAT
0,0.22557,0.239149,1195.0,#NULL!,#NULL!,#NULL!,#NULL!,03 - CA - 50 - 3 - 01,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.22557,0.239149,1195.0,#NULL!,#NULL!,#NULL!,#NULL!,03 - CA - 50 - 3 - 02,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.22557,0.239149,1195.0,#NULL!,#NULL!,#NULL!,#NULL!,03 - CA - 50 - 3 - 03,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.22557,0.239149,1195.0,#NULL!,#NULL!,#NULL!,#NULL!,03 - CA - 50 - 3 - 04,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.22557,0.239149,1195.0,#NULL!,#NULL!,#NULL!,#NULL!,03 - CA - 50 - 3 - 05,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Create an overview table grouped by State
per_state = dui_data_complete.set_index("State").groupby(["State"])
per_state.head()

In [ ]:
#Declare columns/variables to keep and create a summary dataframe
#links to data dictionaries for dui studies (help decided relevant columns)
#https://www.nhtsa.gov/sites/nhtsa.dot.gov/files/documents/2007_nrsdatadictionary.pdf
#https://www.nhtsa.gov/sites/nhtsa.dot.gov/files/documents/nrs_2013datadictionary.pdf

timeday
STATE
NRS_AREA
DATE

dui_summary = pd.DataFrame({})

dui_summary.head()

In [ ]:
#census data link to retrieve population information per state
#https://www.census.gov/quickfacts/fact/table/US/PST040219